# LGBM Model development - CV one model per fold - optim NelderMead

In [1]:
import pandas as pd
import seaborn as sns
import openpyxl
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout,LSTM
from keras import regularizers, Input

In [2]:
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna
from vmdpy import VMD

In [3]:
from scipy.optimize import minimize

In [4]:
from Functions.helper_functions import * 

In [5]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
np.random.seed(42)

In [7]:
  ###################################
# TensorFlow wizardry
config = tf.compat.v1.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
# Only allow a total of half the GPU memory to be allocatedC
config.gpu_options.per_process_gpu_memory_fraction = 0.5
# Create a session with the above options specified.
tf.compat.v1.Session(config=config)
###################################

# Data

In [8]:
train_wp1 = pd.read_csv('Data/Preprocessing/WP1_train_preprocessed.csv', sep=',')
train_wp2 = pd.read_csv('Data/Preprocessing/WP2_train_preprocessed.csv', sep=',')
train_wp3 = pd.read_csv('Data/Preprocessing/WP3_train_preprocessed.csv', sep=',')
train_wp4 = pd.read_csv('Data/Preprocessing/WP4_train_preprocessed.csv', sep=',')
train_wp5 = pd.read_csv('Data/Preprocessing/WP5_train_preprocessed.csv', sep=',')
train_wp6 = pd.read_csv('Data/Preprocessing/WP6_train_preprocessed.csv', sep=',')

In [9]:
test_wp1 = pd.read_csv('Data/Preprocessing/WP1_test_preprocessed.csv', sep=',')
test_wp2 = pd.read_csv('Data/Preprocessing/WP2_test_preprocessed.csv', sep=',')
test_wp3 = pd.read_csv('Data/Preprocessing/WP3_test_preprocessed.csv', sep=',')
test_wp4 = pd.read_csv('Data/Preprocessing/WP4_test_preprocessed.csv', sep=',')
test_wp5 = pd.read_csv('Data/Preprocessing/WP5_test_preprocessed.csv', sep=',')
test_wp6 = pd.read_csv('Data/Preprocessing/WP6_test_preprocessed.csv', sep=',')
test_dates = pd.read_csv('Data/Initial/test.csv', sep=',').date.values

In [10]:
to_drop = ['date','wd','forecast_time', 'forecast', "forecast_dist"]

In [11]:
u_to_drop = [
    'u_T_1', 'u_T_2', 'u_T_3', 'u_T_4', 'u_T_5', 'u_T_6', 
    'u_T_2_mean', 'u_T_3_mean', 'u_T_4_mean', 'u_T_5_mean', 'u_T_6_mean', 'u_T_7_mean',
    'u_T_8_mean', 'u_T_9_mean', 'u_T_10_mean', 'u_T_11_mean', 'u_T_12_mean','u_T_24_mean',
    'u_T_2_std', 'u_T_4_std', 'u_T_5_std', 'u_T_6_std',
    'u_T_2_median', 'u_T_3_median', 'u_T_4_median', 'u_T_5_median', 'u_T_6_median', 'u_T_12_median','u_T_24_median', 'u_T_36_median',
    'u_T_2_max', 'u_T_3_max', 'u_T_4_max', 'u_T_5_max', 'u_T_6_max', 'u_T_12_max',
    'u_T_2_min', 'u_T_3_min', 'u_T_4_min', 'u_T_5_min', 'u_T_6_min', 'u_T_12_min',
    'u2_T_1', 'u2_T_2', 'u2_T_3', 'u2_T_4', 'u2_T_5', 'u2_T_6', 
    'u2_T_2_mean', 'u2_T_3_mean', 'u2_T_4_mean', 'u2_T_5_mean', 'u2_T_6_mean', 'u2_T_7_mean',
    'u2_T_8_mean', 'u2_T_9_mean', 'u2_T_10_mean', 'u2_T_11_mean', 'u2_T_12_mean','u2_T_24_mean',
    'u2_T_2_std', 'u2_T_4_std', 'u2_T_5_std', 'u2_T_6_std', 'u2_T_24_std',
    'u2_T_2_median', 'u2_T_3_median', 'u2_T_4_median', 'u2_T_5_median', 'u2_T_6_median', 'u2_T_12_median',
    'u2_T_2_max','u2_T_3_max', 'u2_T_4_max','u2_T_5_max', 'u2_T_6_max', 'u2_T_12_max',
    'u2_T_2_min', 'u2_T_3_min', 'u2_T_4_min', 'u2_T_5_min', 'u2_T_6_min',
    'u2_T_12', 'u2_T_36_mean', 'u2_T_36_std', 'u2_T_24_median', 'u2_T_24_max',
    'u_T_36_mean','u_T_12','u_T_24_max','u2_T_36_median','u_T_24_min'
]
ws_to_drop = [
    'ws_T_1', 'ws_T_2', 'ws_T_3', 'ws_T_4', 'ws_T_5', 'ws_T_6', 'ws_T_7', 'ws_T_8', 'ws_T_10','ws_T_11', 'ws_T_12',
    'ws_T_2_mean', 'ws_T_3_mean', 'ws_T_4_mean', 'ws_T_5_mean', 'ws_T_6_mean', 'ws_T_7_mean', 'ws_T_8_mean', 'ws_T_9_mean', 
    'ws_T_10_mean', 'ws_T_11_mean', 'ws_T_12_mean', 'ws_T_24_mean', 
    'ws_T_2_std', 'ws_T_3_std', 'ws_T_4_std', 'ws_T_5_std', 
    'ws_T_2_median', 'ws_T_3_median', 'ws_T_4_median', 'ws_T_5_median', 'ws_T_6_median',
    'ws_T_12_median', 'ws_T_24_median', 'ws_T_36_median',
    'ws_T_2_max', 'ws_T_3_max', 'ws_T_4_max', 'ws_T_5_max','ws_T_6_max', 'ws_T_12_max',
     'ws_T_2_min', 'ws_T_3_min', 'ws_T_4_min', 'ws_T_5_min', 'ws_T_6_min', 'ws_T_12_min','ws_T_24_max','ws_T_24_min'
]

v_to_drop = [
    'v_T_1', 'v_T_2', 'v_T_3', 'v_T_4', 'v_T_5', 'v_T_6', 
    'v_T_2_mean', 'v_T_3_mean', 'v_T_4_mean', 'v_T_5_mean', 'v_T_6_mean', 'v_T_7_mean',
    'v_T_8_mean', 'v_T_9_mean', 'v_T_10_mean', 'v_T_11_mean', 'v_T_12_mean', 'v_T_24_mean','v_T_36_mean',
    'v_T_3_std', 'v_T_4_std', 'v_T_5_std','v_T_6_std','v_T_24_std', 'v_T_36_median',
    'v_T_2_median', 'v_T_3_median', 'v_T_4_median', 'v_T_5_median', 'v_T_6_median', 
    'v_T_2_max', 'v_T_3_max', 'v_T_4_max', 'v_T_5_max', 'v_T_6_max', 'v_T_12_max', 
    'v_T_2_min', 'v_T_3_min', 'v_T_4_min', 'v_T_5_min', 'v_T_6_min', 'v_T_12_min', 
    'v_T_36_min', 'v_T_36', 'v_T_24_max',  'v_T_12_median', 'v_T_24_median',
]

wd_to_drop = [
    'coswd_1', 'coswd_2', 'coswd_3', 'coswd_4', 'coswd_5', 'coswd_6',
    'coswd_2_mean', 'coswd_3_mean', 'coswd_4_mean', 'coswd_5_mean', 'coswd_6_mean', 'coswd_7_mean', 
    'coswd_8_mean', 'coswd_9_mean', 'coswd_10_mean', 'coswd_11_mean', 'coswd_12_mean', 'coswd_24_mean', 
    'coswd_3_std', 'coswd_4_std','coswd_5_std','coswd_2_median', 'coswd_3_median','coswd_4_median', 
    'coswd_5_median', 'coswd_6_median', 'coswd_36_median', 'coswd_24_median', 'coswd_12_median',
    'coswd_2_max', 'coswd_3_max', 'coswd_4_max', 'coswd_5_max', 'coswd_6_max', 'coswd_12_max', 'coswd_24_max',
    'coswd_2_min', 'coswd_3_min', 'coswd_4_min', 'coswd_5_min', 'coswd_6_min', 'coswd_12_min', 'coswd_24_min',
    'ws_T_36_max', 'ws_T_36_min', 'coswd_12', 'coswd_24'
]

other_to_drop = [
    'cos_day', 'u', 'v'
]

feature_corr = u_to_drop+ws_to_drop+v_to_drop+wd_to_drop+other_to_drop
#to_drop = to_drop+feature_corr

# DNN functions

In [15]:
#Architecture du modèle:
def build_model(data,n_neurons,nlayers,dropout,style,n):
    layers=[]
    if style=='linear':
        for _ in range(nlayers):
            if len(layers)==0:
                layers.append(Dense(n_neurons,activation='relu',input_shape=(data.shape[1],)))
                layers.append(Dropout(dropout))
            else:
                layers.append(Dense(n_neurons,activation='relu'))
                layers.append(Dropout(dropout))
    else:
        for i in range(nlayers):
            if len(layers)==0:
                layers.append(Dense(n_neurons,activation='relu',input_shape=(data.shape[1],)))
                layers.append(Dropout(dropout))
            elif (len(layers)+1)%nlayers==0:
                layers.append(Dense(n_neurons,activation='relu'))
                layers.append(Dropout(dropout))                
            else:
                layers.append(Dense(int(n_neurons*n),activation='relu'))
                layers.append(Dropout(dropout))
        
    
      
    layers.append(Dense(1,activation='sigmoid'))

    #with tf.device('/device:GPU:0'):
    model = Sequential(layers)
    model.compile(loss='mae',
                optimizer='rmsprop',
                metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

In [12]:
#Architecture du modèle:
def build_model_v2(data,n_neurons,nlayers,dropout,style,n,norm):
    input_tensor=Input(shape=(data.shape[1],))
    if norm==1:
        if style == 'cyclic':
            for i in range(nlayers):
                if i==0:
                    x = layers.Dense(n_neurons,activation='relu')(input_tensor)
                    x=layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                elif i%2!=0 and i!=0:
                    x = layers.Dense(int(n_neurons*n),activation='relu')(x)
                    x=layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                else:
                    x = layers.Dense(n_neurons,activation='relu')(x)
                    x=layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
        elif style == 'linear':
            for i in range(nlayers):
                if i==0:
                    x = layers.Dense(n_neurons,activation='relu')(input_tensor)
                    x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                else:
                    x = layers.Dense(n_neurons,activation='relu')(x)
                    x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
        elif style=='multiply':
            for i in range(nlayers):
                if i==0:
                    x = layers.Dense(n_neurons,activation='relu')(input_tensor)
                    x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                else:
                    x = layers.Dense(int(n_neurons*n),activation='relu')(x)
                    x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
        elif style=='power':
            for i in range(nlayers):
                if i==0:
                    x = layers.Dense(n_neurons,activation='relu')(input_tensor)
                    x=layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                else:
                    x = layers.Dense(int(n_neurons*(n*i)),activation='relu')(x)
                    x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
    else:
        if style == 'cyclic':
            for i in range(nlayers):
                if i==0:
                    x = layers.Dense(n_neurons,activation='relu')(input_tensor)
                    #x=layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                elif i%2!=0 and i!=0:
                    x = layers.Dense(int(n_neurons*n),activation='relu')(x)
                    #x=layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                else:
                    x = layers.Dense(n_neurons,activation='relu')(x)
                    #x=layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
        elif style == 'linear':
            for i in range(nlayers):
                if i==0:
                    x = layers.Dense(n_neurons,activation='relu')(input_tensor)
                    #x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                else:
                    x = layers.Dense(n_neurons,activation='relu')(x)
                    #x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
        elif style=='multiply':
            for i in range(nlayers):
                if i==0:
                    x = layers.Dense(n_neurons,activation='relu')(input_tensor)
                    #x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                else:
                    x = layers.Dense(int(n_neurons*n),activation='relu')(x)
                    #x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
        elif style=='power':
            for i in range(nlayers):
                if i==0:
                    x = layers.Dense(n_neurons,activation='relu')(input_tensor)
                    #x=layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
                else:
                    x = layers.Dense(int(n_neurons*(n*i)),activation='relu')(x)
                    #x = layers.BatchNormalization()(x)
                    x=layers.Dropout(dropout)(x)
    
    
        
        
    
    output_tensor=layers.Dense(1,activation='sigmoid')(x)
    model = Model(input_tensor,output_tensor)
    
    model.compile(loss='mae',
                optimizer='rmsprop',
                metrics=[tf.keras.metrics.RootMeanSquaredError()])
   
    return model

In [13]:
from keras import backend as k

In [14]:
def hyperparametrization(trial, train_x, test_x, train_y, test_y):
    epochs = trial.suggest_int('epochs',1,50)
    n_neurons = trial.suggest_int('n_neurons',10,200)
    nlayers = trial.suggest_int('nlayer',1,20)
    dropout = trial.suggest_categorical('dropout',[0,0.1,0.25,0.5,0.3,0.4,0.2])
    style = trial.suggest_categorical('style',['linear','cyclic','power','multiply'])
    n = trial.suggest_int('n',1,4)
    norm = trial.suggest_categorical('norm',[0,1])
  
      
    tf_train = tf.data.Dataset.from_tensor_slices((train_x,train_y)).batch(36).prefetch(1)
    tf_val = tf.data.Dataset.from_tensor_slices((test_x,test_y)).batch(48).prefetch(1)
    print('#epochs:',epochs,'#layers:',nlayers,'#neurons',n_neurons)
    model = build_model_v2(data=train_x,n_neurons=n_neurons,nlayers=nlayers,dropout=dropout,style=style,n=n,norm=norm)
    model.fit(tf_train,verbose=0,epochs=epochs)   
   
    
    return model.evaluate(tf_val,verbose=0)[1]

In [15]:
def create_lst_model(cv,x,nlayers,dropout,n_neurons):
    model_1=[]
    model_2=[]
    model_3=[]
    model_4=[]
    model_5=[]
    model_6=[]
    for i in range(cv):
        model=build_model(data=x,
                  n_neurons=n_neurons,
                  nlayers=nlayers,
                  dropout=dropout,
                  style='other',
                  n=2)
        model_1+=[model]
        model_2+=[model]
        model_3+=[model]
        model_4+=[model]
        model_5+=[model]
        model_6+=[model]
    return [model_1,model_2,model_3,model_4,model_5,model_6]

In [43]:
def create_lst_model_v2(cv,x,nlayers,dropout,n_neurons,style,n,norm):
    model_1=[]
    model_2=[]
    model_3=[]
    model_4=[]
    model_5=[]
    model_6=[]
    for i in range(cv):
        model=build_model_v2(data=x,
                  n_neurons=n_neurons,
                  nlayers=nlayers,
                  dropout=dropout,
                  style=style,
                  n=n,
                          norm=norm)
        model_1+=[model]
        model_2+=[model]
        model_3+=[model]
        model_4+=[model]
        model_5+=[model]
        model_6+=[model]
    return [model_1,model_2,model_3,model_4,model_5,model_6]

# DATA

In [16]:
wp1_X = train_wp1[[c for c in train_wp1 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X1 = wp1_X.drop('wp', axis=1)
y1 = wp1_X['wp']

In [17]:
wp2_X = train_wp2[[c for c in train_wp2 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X2 = wp2_X.drop('wp', axis=1)
y2 = wp2_X['wp']

In [18]:
wp3_X = train_wp3[[c for c in train_wp3 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X3 = wp3_X.drop('wp', axis = 1)
y3 = wp3_X['wp']


In [19]:
wp4_X = train_wp4[[c for c in train_wp4 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X4 = wp4_X.drop('wp', axis = 1)
y4 = wp4_X['wp']

In [20]:
wp5_X = train_wp5[[c for c in train_wp5 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X5 = wp5_X.drop('wp', axis = 1)
y5 = wp5_X['wp']

In [21]:
wp6_X = train_wp6[[c for c in train_wp6 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X6 = wp6_X.drop('wp', axis = 1)
y6 = wp6_X['wp']

# Creating the 8 dataset per WP

In [22]:
from Functions.preprocessing import *

In [23]:
lst_X1_train, lst_y1_train,lst_X1_test,lst_y1_test =  splitting_train_test_forecast(df_wp=train_wp1)

In [24]:
for i in range(len(lst_X1_train)):
    lst_X1_train[i] = lst_X1_train[i].drop(to_drop,axis=1)
    lst_X1_test[i] = lst_X1_test[i].drop(to_drop,axis=1)

In [30]:
lst_X2_train, lst_y2_train,lst_X2_test,lst_y2_test =  splitting_train_test_forecast(df_wp=train_wp2)

In [31]:
for i in range(len(lst_X1_train)):
    lst_X2_train[i] = lst_X2_train[i].drop(to_drop,axis=1)
    lst_X2_test[i] = lst_X2_test[i].drop(to_drop,axis=1)

In [32]:
lst_X3_train, lst_y3_train,lst_X3_test,lst_y3_test =  splitting_train_test_forecast(df_wp=train_wp3)

In [33]:
for i in range(len(lst_X3_train)):
    lst_X3_train[i] = lst_X3_train[i].drop(to_drop,axis=1)
    lst_X3_test[i] = lst_X3_test[i].drop(to_drop,axis=1)

In [34]:
lst_X4_train, lst_y4_train,lst_X4_test,lst_y4_test =  splitting_train_test_forecast(df_wp=train_wp4)

In [35]:
for i in range(len(lst_X4_train)):
    lst_X4_train[i] = lst_X4_train[i].drop(to_drop,axis=1)
    lst_X4_test[i] = lst_X4_test[i].drop(to_drop,axis=1)

In [36]:
lst_X5_train, lst_y5_train,lst_X5_test,lst_y5_test =  splitting_train_test_forecast(df_wp=train_wp5)

In [37]:
for i in range(len(lst_X5_train)):
    lst_X5_train[i] = lst_X5_train[i].drop(to_drop,axis=1)
    lst_X5_test[i] = lst_X5_test[i].drop(to_drop,axis=1)

In [38]:
lst_X6_train, lst_y6_train,lst_X6_test,lst_y6_test =  splitting_train_test_forecast(df_wp=train_wp6)

In [39]:
for i in range(len(lst_X6_train)):
    lst_X6_train[i] = lst_X6_train[i].drop(to_drop,axis=1)
    lst_X6_test[i] = lst_X6_test[i].drop(to_drop,axis=1)

In [40]:
lst_X = [X1, X2, X3, X4, X5, X6]
lst_Y = [y1, y2, y3, y4, y5, y6]

In [41]:
lst_X_trains_split = [lst_X1_train,lst_X2_train,lst_X3_train,lst_X4_train,lst_X5_train,lst_X6_train]
lst_Y_trains_split = [lst_y1_train,lst_y2_train,lst_y3_train,lst_y4_train,lst_y5_train,lst_y6_train]
lst_X_test_split = [lst_X1_test,lst_X2_test,lst_X3_test,lst_X4_test,lst_X5_test,lst_X6_test]
lst_Y_test_split = [lst_y1_test,lst_y2_test,lst_y3_test,lst_y4_test,lst_y5_test,lst_y6_test]

## Optuna

In [ ]:
params_1=[]
for x_train,y_train,x_test,y_test in zip(lst_X1_train,lst_y1_train,lst_X1_test,lst_y1_test):
    def objective_wp1(trial):
        
        return hyperparametrization(trial, train_x=x_train, test_x=x_test, train_y=y_train, test_y=y_test)
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_wp1, n_trials=50)
#write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
    best_trial = study.best_trial.params
    print(best_trial)
    params_1.append(best_trial)

In [29]:
params_1

[{'epochs': 45,
  'n_neurons': 118,
  'nlayer': 13,
  'dropout': 0.1,
  'style': 'cyclic',
  'n': 2,
  'norm': 0}]

# Predictions

In [49]:
#tf.debugging.set_log_device_placement(True)
#with tf.device('/device:GPU:0'):
lst_model = create_lst_model_v2(cv=7,x=X1,nlayers=5,dropout=0.1,n_neurons=64,style='linear',n=2,norm=0)
#print(lst_model[0][0].summary())
#print('---check eights on first layer-----')
#print(lst_model[0][0].get_weights()[0])
lst_predictions_all=[]
t=1
for model,x_all,y_all,x,y,x_test,y_test in zip(lst_model, lst_X,lst_Y, lst_X_trains_split, lst_Y_trains_split, lst_X_test_split, lst_Y_test_split):
    lst_predictions=[]
    print('--------WP n°',t,'--------')
    #scaler_x = MinMaxScaler(feature_range=(0,1))
    for i in range(7):
        print('--- set n°',i,'-----')
        clf=model[i]
        #x_train_scaled = scaler_x.fit_transform(x[i])
        #x_test_scaled=scaler_x.transform(x_test[i])
        #x_all_scaled=scaler_x.transform(x_all)
        #scaler_name = 'scaler_{0}_{1}'.format(t,i)
        tf_train = tf.data.Dataset.from_tensor_slices((x[i],y[i])).batch(36).prefetch(1)
        tf_val = tf.data.Dataset.from_tensor_slices((x_test[i],y_test[i])).batch(48).prefetch(1)
        tf_all = tf.data.Dataset.from_tensor_slices((x_all,y_all)).batch(84).prefetch(1)
        print('-----fiting----')
        start_time = time.time()
        clf.fit(tf_train,  verbose=0, epochs=10) #,
        print("--- %s seconds ---" % (time.time() - start_time))
        print("Evaluate on test data")
        results = clf.evaluate(tf_val,verbose=0) # 
        
        print("test loss, test acc:", results)
        lst_predictions.append(clf.predict(tf_all))
        #pickle.dump(scaler_x,open(scaler_name,"wb"))
        print("--- %s seconds ---" % (time.time() - start_time))
    lst_predictions_all.append(lst_predictions)
    t+=1

--------WP n° 1 --------
--- set n° 0 -----
-----fiting----
--- 70.74976062774658 seconds ---
Evaluate on test data
test loss, test acc: [0.10503058135509491, 0.15564337372779846]
--- 75.4876537322998 seconds ---
--- set n° 1 -----
-----fiting----
--- 84.18728137016296 seconds ---
Evaluate on test data
test loss, test acc: [0.10395887494087219, 0.15179672837257385]
--- 88.0834710597992 seconds ---
--- set n° 2 -----
-----fiting----
--- 68.1334478855133 seconds ---
Evaluate on test data
test loss, test acc: [0.10423261672258377, 0.15388864278793335]
--- 74.37038493156433 seconds ---
--- set n° 3 -----
-----fiting----
--- 73.54607820510864 seconds ---
Evaluate on test data
test loss, test acc: [0.1062958836555481, 0.1557742804288864]
--- 76.94760155677795 seconds ---
--- set n° 4 -----
-----fiting----
--- 72.72862195968628 seconds ---
Evaluate on test data
test loss, test acc: [0.11008834093809128, 0.15697148442268372]
--- 76.34928512573242 seconds ---
--- set n° 5 -----
-----fiting----


In [50]:
weights_all=[]
for pred,y in zip(lst_predictions_all,lst_Y):
    weights=[]
    for i in range(7):
        weights.append(0)
        
    def mae_func(weights):
        #final_prediction=0
        for i in range(len(weights)):
            if i==0:
                final_prediction = weights[i]*pred[i]
            else:
                final_prediction += weights[i]*pred[i]
        return mean_absolute_error(y, final_prediction)
    res = minimize(mae_func, weights, method='Nelder-Mead')
    weights_all.append(res['x'])
        
    

In [51]:
np.shape(weights_all)

(6, 7)

In [52]:
to_drop_test = ['date','wd','forecast_time', 'forecast', "forecast_dist", 'wp']#+feature_corr
def make_prediction_dataset(test, to_drop=to_drop_test):
    test_to_predict = test.dropna(subset=['ws','u','v'], how = 'any') # keeps only lines with u,v,ws,wd
    test_to_predict = test_to_predict[test_to_predict['wp'].isna()] # keeps only lines with no wp
    test_to_predict = test_to_predict.sort_values(by=['date', 'forecast_time'], ascending = [True, False]).drop_duplicates(subset='date')
    test_to_predict = test_to_predict.drop(to_drop, axis = 1)
    return test_to_predict

In [53]:
lst_tests = []
for test in [test_wp1, test_wp2, test_wp3, test_wp4, test_wp5, test_wp6]:
    test = make_prediction_dataset(test)
    lst_tests.append(test)

In [54]:
lst_final_prediction=[]
for weights,model,test,x_train,y_train in zip(weights_all,lst_model,lst_tests,lst_X_trains_split,lst_Y_trains_split):
    t=1
    for i in range(7):
        clf=model[i]
        #scaler = pickle.load(open('scaler_{0}_{1}'.format(t,i),"rb"))
        #x_test=scaler.transform(test)
        #clf.fit(x_train[i],y_train[i],verbose=0,batch_size=36)
        x_test = tf.data.Dataset.from_tensor_slices((test)).batch(84).prefetch(1)
        if i==0:
            y_pred=clf.predict(x_test,batch_size=84)*weights[i]
        else:
            y_pred+=clf.predict(x_test,batch_size=84)*weights[i]
    lst_final_prediction.append(y_pred)
    t+=1
        

In [55]:
np.shape(lst_final_prediction[0].reshape(7440,))

(7440,)

In [56]:
df_predictions = pd.DataFrame({
        'date': test_dates,
        'wp1': lst_final_prediction[0].reshape(7440,),
        'wp2': lst_final_prediction[1].reshape(7440,),
        'wp3': lst_final_prediction[2].reshape(7440,),
        'wp4': lst_final_prediction[3].reshape(7440,),
        'wp5': lst_final_prediction[4].reshape(7440,),
        'wp6': lst_final_prediction[5].reshape(7440,),        
    })

In [57]:
nb_sub = 50
model = "dnn_7models_traintestsplit"
prepro = 'RobustScaler'
postpro = "Prediction limited by 0-1"

In [58]:
df_predictions.head()

,date,wp1,wp2,wp3,wp4,wp5,wp6
0,2011010101,0.529483,0.270229,1.425836e-07,0.340227,0.446056,0.437530
1,2011010102,0.520036,0.312680,-1.659469e-07,0.341136,0.436530,0.417496
2,2011010103,0.524673,0.330738,7.823510e-05,0.340288,0.464695,0.426882
3,2011010104,0.541555,0.327666,3.110182e-02,0.372731,0.505008,0.428434
4,2011010105,0.564103,0.308715,1.899543e-01,0.407729,0.550857,0.425569


In [59]:
# df_predictions.to_csv('Predictions/submission_nb_10_full_maxabs-lgbm-featselect.csv', index=False, sep=';')
df_predictions.to_csv(f'Predictions/submission_nb_{nb_sub}_{model}.csv', index=False, sep=';')